In [ ]:
# input: 1 ảnh png jpg chuyển thành Tensor
# output: 1 list các bounding boxes cho reID model 
from PIL import Image
image = Image.open("0.jpg")

from ultralytics import YOLO
# Load the YOLOv8 model
model = YOLO('yolov8x.pt')
predict = model.predict(source=image)

In [ ]:
# print(type(predict[0]))
# print(len(predict))
# print(predict)

In [ ]:
# Lấy thông tin bounding boxes từ results
# print(dir(predict[0]))
# boxes = predict[0].orig_img
# print(boxes)

# boxes = predict[0].boxes
# print(type(boxes))
# print(boxes)

# print(len(boxes))           # height
# print(len(boxes[0]))        # width
# print(len(boxes[0][0]))     # RGB

In [ ]:
bounding_boxes = predict[0].boxes.xywh
print(bounding_boxes)

In [7]:
from ultralytics import YOLO
yolo_model = YOLO('yolov8x.pt')

import torchreid
reid_model = torchreid.models.build_model(
    name='mudeep',
    num_classes=1000,
    pretrained=True
)

from PIL import Image
import torch
from torchvision.transforms import functional as F

def detect_objects(image_path, yolo_model):
    # Load ảnh
    img = Image.open(image_path).convert('RGB')

    # Sử dụng YOLO để xác định vùng chứa object
    results = yolo_model(img)

    # Lấy thông tin về object và bounding box
    boxes = results[0].boxes.xywh   # tensor

    # Cắt và lưu các vùng chứa object
    object_images = []
    for box in boxes:
        xmin, ymin, width, height = map(int, box)
        xmax = xmin + width
        ymax = ymin + height
        object_img = img.crop((xmin, ymin, xmax, ymax))
        object_images.append(object_img)

    return object_images

from torchvision import transforms
def extract_features(image, reid_model):
    # Load ảnh và tiền xử lý nó để phù hợp với mô hình
    img = image.convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize((256, 128)),
        transforms.ToTensor(),
    ])
    img = preprocess(img).unsqueeze(0)

    # Đưa ảnh qua mô hình để lấy đặc trưng
    with torch.no_grad():
        features = reid_model.featuremaps(img)

    # return features.squeeze().numpy()
    return features

# def main():
#     from sklearn.metrics.pairwise import cosine_similarity
#     # Đường dẫn đến ảnh 1 và ảnh 2
#     image_path_1 = '0.jpg'
#     image_path_2 = '1.jpg'

#     # Dùng YOLO để xác định vùng chứa object trên ảnh 1 và ảnh 2
#     object_images_1 = detect_objects(image_path_1, yolo_model)
#     object_images_2 = detect_objects(image_path_2, yolo_model)

#     # Trích xuất vector đặc trưng từ reid_model cho từng vùng chứa object
#     features_1 = [extract_features(img, reid_model) for img in object_images_1]
#     features_2 = [extract_features(img, reid_model) for img in object_images_2]

#     # # So sánh các vector đặc trưng
#     # for i, feature_1 in enumerate(features_1):
#     #     for j, feature_2 in enumerate(features_2):
#     #         # Lấy mảng numpy từ tensor và làm phẳng
#     #         feature_1 = feature_1[0].ravel()
#     #         feature_2 = feature_2[0].ravel()

#     #         # Sử dụng phương pháp so sánh phù hợp (ví dụ: cosine similarity)
#     #         similarity_score = cosine_similarity(feature_1.reshape(1, -1), feature_2.reshape(1, -1))

#     #         # In kết quả
#     #         print(f'Similarity between object {i+1} in image 1 and object {j+1} in image 2: {similarity_score[0, 0]}')

#     print(features_1)
#     print(features_2)
#     print(type(features_1))
#     print(len(features_1))
#     print(len(features_2))


# if __name__ == "__main__":
#     main()

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
# Đường dẫn đến ảnh 1 và ảnh 2
image_path_1 = '0.jpg'
image_path_2 = '1.jpg'
# Dùng YOLO để xác định vùng chứa object trên ảnh 1 và ảnh 2
object_images_1 = detect_objects(image_path_1, yolo_model)
object_images_2 = detect_objects(image_path_2, yolo_model)


0: 320x640 4 persons, 1 car, 2508.0ms
Speed: 5.7ms preprocess, 2508.0ms inference, 1.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 persons, 2444.3ms
Speed: 5.0ms preprocess, 2444.3ms inference, 2.6ms postprocess per image at shape (1, 3, 320, 640)


In [13]:
print(type(object_images_1[0]))
print(object_images_1[0].size)

<class 'PIL.Image.Image'>
(845, 594)


In [ ]:
# Trích xuất vector đặc trưng từ reid_model cho từng vùng chứa object
features_1 = [extract_features(img, reid_model) for img in object_images_1]
features_2 = [extract_features(img, reid_model) for img in object_images_2]

In [ ]:
from siamese_network import SiameseNetwork
sinetwork = SiameseNetwork()
# compare = [sinetwork.forward(i, j) for i in object_images_1 for j in object_images_2]

# Định nghĩa một hàm chuyển đổi hình ảnh thành tensor
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Duyệt qua từng cặp hình ảnh và tính toán độ tương đồng
compare = [
    sinetwork.forward(transform(i).unsqueeze(0), transform(j).unsqueeze(0))
    for i, j in zip(object_images_1, object_images_2)
]

In [ ]:
threshold = 0.7

for i in range(len(features_1)):
    for j in range(len(features_2)):
        # features_1 và features_2 là các đặc trưng được trích xuất từ hai ảnh
        similarity_score = cosine_similarity(features_1[i].reshape(1, -1), features_2[j].reshape(1, -1))

        # In kết quả
        print(f'features_1[{i}] and features_2[{j}]: {similarity_score[0, 0]}')

        if similarity_score >= threshold:
            print(f"features_1[{i}] is similar to features_2[{j}].")

In [ ]:
print(features_1[0].shape)
print(features_2[0].shape)

In [ ]:
for feature in features_1:
    print(feature)
    print(len(feature))
    print(type(feature))

In [ ]:
print(type(features_1))
print(features_1[0][0])
print(features_1[0].shape)

In [ ]:
print(type(features_2))
print(features_2[0][0])
print(features_2[0].shape)

In [ ]:
print(len(features_1), len(features_2))

In [ ]:
print(object_images_1)
print(len(object_images_1))
print(type(object_images_1))

In [15]:
# !pip install scikit-learn
# !pip install fastreid